In [1]:
# import package
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
import matplotlib.image as mpimg

In [2]:
# add to library search path
sys.path.append("/home/mao/Documents/Github/models/research")
sys.path.append('/home/mao/Documents/Github/models/research/object_detection')

from object_detection.utils import ops as utils_ops

from utils import label_map_util
from utils import visualization_utils as vis_util

In [3]:
# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = '/home/mao/Documents/pre_models/ssd_mobilenet_v1_coco_2017_11_17/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/home/mao/Documents/Github/models/research/object_detection/data/mscoco_label_map.pbtxt'

# number of classes
NUM_CLASSES = 90

In [4]:
# import .pb graph 
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [5]:
# map classID to className
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [7]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = '/home/mao/Documents/Github/models/research/object_detection/test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.jpg'.format(i)) for i in range(1, 3) ]
# Size, in inches, of the output images.
IMAGE_SIZE = (20, 20)

In [8]:
# run inference
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [9]:
TEST_IMAGE_PATHS

['/home/mao/Documents/Github/models/research/object_detection/test_images/image1.jpg',
 '/home/mao/Documents/Github/models/research/object_detection/test_images/image2.jpg']

In [10]:
test_path = TEST_IMAGE_PATHS[0]

In [11]:
image = mpimg.imread(test_path)

In [12]:
image.shape

(636, 1024, 3)

In [14]:
output_dict = run_inference_for_single_image(image, detection_graph)

In [15]:
output_dict

{'detection_boxes': array([[3.90841365e-02, 1.92150474e-02, 8.72103572e-01, 3.15773487e-01],
        [1.09515011e-01, 4.02835608e-01, 9.24646080e-01, 9.73047853e-01],
        [5.07123828e-01, 3.85651469e-01, 8.76479506e-01, 7.03940868e-01],
        [6.38731122e-02, 2.81166583e-01, 9.16385174e-01, 8.24097157e-01],
        [2.15276748e-01, 3.96740973e-01, 9.12904143e-01, 6.79073751e-01],
        [5.27002156e-01, 3.72713208e-01, 8.83502662e-01, 6.94296718e-01],
        [0.00000000e+00, 5.25425494e-01, 2.53364950e-01, 7.16124356e-01],
        [7.37802982e-02, 2.98963934e-01, 9.20516491e-01, 9.38026905e-01],
        [5.31044006e-01, 3.84130090e-01, 9.15436625e-01, 8.53949070e-01],
        [1.45327359e-01, 3.87902379e-01, 9.06316161e-01, 9.78849053e-01],
        [2.08420545e-01, 2.78770030e-01, 8.26954961e-01, 6.09079719e-01],
        [3.48854452e-01, 3.05225879e-01, 8.99384499e-01, 7.33766913e-01],
        [7.35044479e-03, 5.13691902e-02, 6.02022469e-01, 8.20266247e-01],
        [7.01457560

In [17]:
rclasses = output_dict['detection_classes']
rscores = output_dict['detection_scores']
rbboxes = output_dict['detection_boxes']

In [18]:
rclasses

array([18, 18, 18, 18, 18, 33,  1, 63, 18, 21,  3, 18,  1, 18, 21, 18, 62,
       18, 18, 62,  1, 62, 18,  3,  1, 21,  1,  1,  1,  1,  1, 18,  1,  1,
        1,  1, 62,  1,  1,  1,  1,  1, 62, 18,  1, 63,  1,  1, 84,  1,  1,
       18,  1,  1, 62, 21, 57, 18, 18, 62, 18,  1,  1,  1,  1,  1,  1,  1,
       18, 62,  1, 88, 47,  1, 47, 62, 84,  1, 18,  1,  1,  1,  1, 84, 18,
       62, 18, 21,  1, 62, 62,  1, 18, 21,  1, 18, 47,  1,  1, 47],
      dtype=uint8)

In [19]:
rclasses.shape

(100,)

In [20]:
rscores

array([0.940691  , 0.9345028 , 0.23088078, 0.22518921, 0.17249939,
       0.13962041, 0.13212018, 0.0997574 , 0.08992735, 0.08888885,
       0.0809183 , 0.07895196, 0.07818312, 0.07565781, 0.07331279,
       0.07074906, 0.06597455, 0.0640453 , 0.06387984, 0.06381167,
       0.06380174, 0.06050131, 0.05898518, 0.05898361, 0.05761391,
       0.05641384, 0.05638783, 0.05500019, 0.05495735, 0.0542639 ,
       0.05425419, 0.0540957 , 0.05335919, 0.05279226, 0.05228501,
       0.05219467, 0.05164067, 0.05066015, 0.05030202, 0.04837739,
       0.04822724, 0.0472066 , 0.04673688, 0.04671786, 0.04644237,
       0.04614897, 0.04420188, 0.04286509, 0.04251253, 0.04174341,
       0.04166571, 0.04137339, 0.04096094, 0.0405353 , 0.03990957,
       0.03988707, 0.0398438 , 0.03930983, 0.03923886, 0.03877974,
       0.03877512, 0.03852981, 0.03816218, 0.03815899, 0.03742454,
       0.0368987 , 0.03677591, 0.03671159, 0.03654773, 0.03635325,
       0.03612373, 0.03610818, 0.0360195 , 0.03586452, 0.03582

In [21]:
len(rclasses)

100

In [ ]:
for i in range(len(rclasses)):
    probability = rscores[i]
    if probability > 0.5:
        